In [11]:
def dict_create(dictionary):
    #stores which characters appear in a given sentence
    sen_char = dict()
    for char in dictionary.keys():
        for i in dictionary[char]["sentences"]:
            if i not in sen_char:
                sen_char[i] = []
            sen_char[i].append(char)
    return sen_char

In [21]:
def events(sen_char, total_sentences,retention=0.5, threshold=0.5, penalty=0.5):
    
    def f1(idx=0):
        event = [idx]
        chars = set(sen_char[idx])
        l = len(chars)
        mov_avg = 1
#         print("\n\n\n")
        while idx<total_sentences-1:
            idx += 1 
            if idx not in sen_char:
                continue
            chars_nxt = set(sen_char[idx])
            p = len(chars.intersection(chars_nxt)) #common characters in both sentences
            n = len(chars_nxt) - p #new characters
            m = l - p #missing characers
            #Calculate the moving average mov_avg
            mov_avg=(1-retention)*((p - penalty*(n+m))/l) + retention*mov_avg
#             mov_avg+=((p -  penalty*(n+m)/l) + retention*mov_avg)*0.5
#             print(mov_avg,p,n,m,l, idx)
            if mov_avg>=threshold:
                event.append(idx)
            else:
                break
        return event, idx
    
    idx = 0
    event_no = 0
    events = dict()
    while idx<total_sentences:
        if idx in sen_char:
            event, idx = f1(idx)
            events[event_no] = event
            event_no += 1
        else:
            idx+=1
        if idx >= total_sentences-1:
            break
#         print(idx, total_sentences-1)
    return events

In [13]:
import pickle
file = open(r'C:\Users\Sourav\Desktop\BTP\btp-1\Event_Segmentation\BTP Code\Panchatantra\the_story_of_the_merchant_son_ann.gpickle', 'rb')
grps = pickle.load(file)
file.close()

In [14]:
sen_char = dict_create(grps)

In [72]:
event_list = events(sen_char,66)

In [73]:
event_list

{0: [0, 1, 3],
 1: [4, 5],
 2: [6, 7],
 3: [8, 9],
 4: [10, 11],
 5: [12],
 6: [13],
 7: [14, 15],
 8: [16, 17, 18, 19, 20],
 9: [21, 22, 23],
 10: [24],
 11: [25, 26],
 12: [27, 28],
 13: [29],
 14: [30, 31, 32],
 15: [33, 34],
 16: [36],
 17: [37, 38],
 18: [39, 41],
 19: [43, 44],
 20: [46, 47],
 21: [48],
 22: [49],
 23: [50],
 24: [51],
 25: [52, 53, 54, 55],
 26: [56, 57, 58],
 27: [59, 60, 61],
 28: [62],
 29: [63],
 30: [64, 65]}

In [8]:
words = """day, year, month, time, morning, noon, night, evening, once_upon_a_time"""
words = words.replace("\n\n","\n")
words = words.replace("\n",",")
words = words.replace(" ","")
word_list = words.split(",")
print(word_list)

['day', 'year', 'month', 'time', 'morning', 'noon', 'night', 'evening', 'once_upon_a_time']


In [9]:
from empath import Empath
lexicon = Empath()
lexicon.create_category("time",word_list)

["day", "night", "week", "afternoon", "evening", "the_day", "morning", "weekend", "month", "afternoon", "year", "a_day", "the_week", "days", "summer", "the_month", "the_next_day", "Sunday", "few_hours", "a_week", "whole_day", "day", "two_days", "one_week", "three_hours", "six_hours", "one_hour", "four_hours", "few_days", "a_few_hours", "the_next_week", "three_days", "the_first_week", "two_weeks", "the_next_month", "noon", "two_hours", "the_third_day", "four_days", "an_hour", "Saturday", "hour", "today", "weeks", "evening", "hours", "Saturday", "eight_hours", "nights", "next_day", "this_week", "Thursday", "Sometime", "evenings", "five_days", "the_morning", "whole_week", "week", "the_next_two_days", "a_month", "Wednesday", "Three_days", "the_last_week", "Friday", "one_night", "the_last_two_weeks", "summers", "seven_hours", "five_hours", "whole_month", "three_weeks", "my_day", "that_day", "months", "weeks", "holidays", "since", "midday", "only_an_hour", "first_night", "the_weekend", "Satu

In [25]:
file = open(r'C:\Users\Sourav\Desktop\BTP\btp-1\Event_Segmentation\BTP Code\Panchatantra\the_story_of_the_merchant_son.txt', errors='ignore')
text = file.read()
file.close()

In [80]:
d = lexicon.analyze("temple", categories=["buildings", "terrain", "living_place", "underground","outdoor", "water_place", "architecture"])
print(d)

{'buildings': 0.0, 'terrain': 0.0, 'living_place': 0.0, 'underground': 0.0, 'outdoor': 0.0, 'water_place': 0.0, 'architecture': 0.0}


In [26]:
from stanza.server import CoreNLPClient
with CoreNLPClient(annotators = ['tokenize','ssplit'],
    memory='5G', be_quiet=True, outputFormat = 'json', max_char_length=500000, timeout=36000000) as client:
    ann = client.annotate(text)

2021-02-23 01:00:48 INFO: Writing properties to tmp file: corenlp_server-1d6ac9ed7b364031.props
2021-02-23 01:00:48 INFO: Starting server with command: java -Xmx5G -cp C:\Users\Sourav\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 36000000 -threads 5 -maxCharLength 500000 -quiet True -serverProperties corenlp_server-1d6ac9ed7b364031.props -annotators tokenize,ssplit -preload -outputFormat serialized


In [74]:
def events_by_location(ann, events):
    lexicon = Empath()
    for event_no in events.keys():
        events2 = dict()
        idx = 0
        for sentence in events[event_no]:
            sentence_for_empath = text[ann.sentence[sentence].characterOffsetBegin:ann.sentence[sentence].characterOffsetEnd].replace(", "," ").replace(".","").replace("-"," ").replace("?","").replace("!","").replace(":"," ")
            places = lexicon.analyze(sentence_for_empath, categories=["buildings", "terrain", "living_place", "underground",
                                                                     "outdoor", "water_place", "architecture"])
            s = 0
            for key in places.keys():
                s+=places[key]
            if s>0:
                idx+=1
                events2["Location "+str(idx)]=[sentence]
            else:
                key = "Location "+str(idx)
                if key not in events2:
                    events2[key] = list()
                events2["Location "+str(idx)].append(sentence)
        events[event_no] = events2.copy()
    return events

In [75]:
events_by_location(ann, event_list)

{0: {'Location 0': [0, 1, 3]},
 1: {'Location 0': [4], 'Location 1': [5]},
 2: {'Location 0': [6, 7]},
 3: {'Location 1': [8, 9]},
 4: {'Location 0': [10, 11]},
 5: {'Location 0': [12]},
 6: {'Location 0': [13]},
 7: {'Location 0': [14], 'Location 1': [15]},
 8: {'Location 0': [16, 17], 'Location 1': [18, 19, 20]},
 9: {'Location 0': [21, 22, 23]},
 10: {'Location 0': [24]},
 11: {'Location 0': [25, 26]},
 12: {'Location 0': [27, 28]},
 13: {'Location 0': [29]},
 14: {'Location 0': [30], 'Location 1': [31], 'Location 2': [32]},
 15: {'Location 0': [33], 'Location 1': [34]},
 16: {'Location 0': [36]},
 17: {'Location 0': [37], 'Location 1': [38]},
 18: {'Location 1': [39], 'Location 2': [41]},
 19: {'Location 0': [43, 44]},
 20: {'Location 0': [46, 47]},
 21: {'Location 0': [48]},
 22: {'Location 0': [49]},
 23: {'Location 0': [50]},
 24: {'Location 1': [51]},
 25: {'Location 0': [52, 53, 54, 55]},
 26: {'Location 0': [56, 57, 58]},
 27: {'Location 0': [59, 60, 61]},
 28: {'Location 0':

In [ ]:
def events_by_time(ann, events):
    lexicon = empath()
    for event_no in events.keys():
        events2 = dict()
        idx = 0
        for sentence in events[event_no]:
            sentence_for_empath = text[ann.sentence[sentence].characterOffsetBegin:ann.sentence[sentence].characterOffsetEnd].replace(", "," ")
            time = lexicon.analyze(sentence_for_empath, categories=["time"])
            s = 0
            for key in time.keys():
                s+=time[key]
            if s>0:
                idx+=1
                events2["Time "+str(idx)]=[sentence]
            else:
                events2["Time "+str(idx)].append(sentence)
        events[event_no] = events2.copy()
    return events

In [81]:
def events_by_location_2(ann):
    lexicon = Empath()
    events2 = dict()
    idx = 0
    for i, sentence in enumerate(ann.sentence):
        sentence_for_empath = text[sentence.characterOffsetBegin:sentence.characterOffsetEnd].replace(", "," ").replace(".","").replace("-"," ").replace("?","").replace("!","").replace(":"," ")
        places = lexicon.analyze(sentence_for_empath, categories=["buildings", "terrain", "living_place", "underground",
                                                                 "outdoor", "water_place", "architecture"])
#         print(sentence_for_empath,"\n",places,"\n\n")
        s = 0
        for key in places.keys():
            s+=places[key]
        if s>0:
            idx+=1
            events2["Location "+str(idx)]=[i]
        else:
            key = "Location "+str(idx)
            if key not in events2:
                events2[key] = list()
            events2["Location "+str(idx)].append(i)
    return events2

In [82]:
events_by_location_2(ann)

{'Location 0': [0, 1, 2, 3, 4],
 'Location 1': [5, 6, 7],
 'Location 2': [8, 9, 10, 11, 12, 13, 14],
 'Location 3': [15, 16, 17],
 'Location 4': [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
 'Location 5': [31],
 'Location 6': [32, 33],
 'Location 7': [34, 35, 36, 37],
 'Location 8': [38],
 'Location 9': [39, 40],
 'Location 10': [41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
 'Location 11': [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64],
 'Location 12': [65]}

In [48]:
ann.sentence

[token {
  word: "Sagardatta"
  value: "Sagardatta"
  before: ""
  after: " "
  originalText: "Sagardatta"
  beginChar: 0
  endChar: 10
  tokenBeginIndex: 0
  tokenEndIndex: 1
  hasXmlContext: false
  isNewline: false
}
token {
  word: "was"
  value: "was"
  before: " "
  after: " "
  originalText: "was"
  beginChar: 11
  endChar: 14
  tokenBeginIndex: 1
  tokenEndIndex: 2
  hasXmlContext: false
  isNewline: false
}
token {
  word: "a"
  value: "a"
  before: " "
  after: " "
  originalText: "a"
  beginChar: 15
  endChar: 16
  tokenBeginIndex: 2
  tokenEndIndex: 3
  hasXmlContext: false
  isNewline: false
}
token {
  word: "merchant"
  value: "merchant"
  before: " "
  after: " "
  originalText: "merchant"
  beginChar: 17
  endChar: 25
  tokenBeginIndex: 3
  tokenEndIndex: 4
  hasXmlContext: false
  isNewline: false
}
token {
  word: "who"
  value: "who"
  before: " "
  after: " "
  originalText: "who"
  beginChar: 26
  endChar: 29
  tokenBeginIndex: 4
  tokenEndIndex: 5
  hasXmlContext